In [ ]:
!gdown --id 1HSh3wkhjgMdjtrkwQUPszRKo_KFF-kGk

Downloading...
From: https://drive.google.com/uc?id=1HSh3wkhjgMdjtrkwQUPszRKo_KFF-kGk
To: /content/human_data.txt
5.55MB [00:00, 175MB/s]


In [ ]:
import pandas as pd
import random
df = pd.read_csv('human_data.txt', sep="\t")
"""df is the variable that stores all our data ( DNA strings here )"""
len(df)

4380

In [ ]:
df.head()

,sequence,class
0,ATGCCCCAACTAAATACTACCGTATGGCCCACCATAATTACCCCCA...,4
1,ATGAACGAAAATCTGTTCGCTTCATTCATTGCCCCCACAATCCTAG...,4
2,ATGTGTGGCATTTGGGCGCTGTTTGGCAGTGATGATTGCCTTTCTG...,3
3,ATGTGTGGCATTTGGGCGCTGTTTGGCAGTGATGATTGCCTTTCTG...,3
4,ATGCAACAGCATTTTGAATTTGAATACCAGACCAAAGTGGATGGTG...,3


In [ ]:
def find_shingles(string, k):
    """
    <h5> A function to find the shingles of our data 
    <ol> It takes in 2 inputs :
    <li> The string , from which it will generate the shingles </li>
    <li> The length of each shingle. It will generate a k-shingle and return it </li>
    </ol>
    It returns all the shingles from its string input
    </h5>
    """
    shingle = set()
    for i in range(len(string)-k+1):
        shingle.add(string[i:i+k])
    return shingle



In [ ]:
y = df['class']

In [ ]:
k = 5
"""This is our global variable for the shingle length. We can change it and make k-shingles accordingly"""
def doc_shingles(df, k):
    """
    <h5>The main function to create the shingles of all our documents
    <ul><li> It takes in dataframe , and separates the df into individual document. </li>
    <li> For each document, it takes in each string and passes it onto the function <i>find_shingles</i> to get the shingle.</li> 
    <li> It also checks to ensure that all the shingles are made correctly and there's no shingle which is empty.</li>
    <li> Once done that, we once again add all the shingles of each document together.</li>
    <li> Finally , we add all the documents back into an array and return this </li>
    </ul> </h5>
    """
    shingle_arr = list()
    shingle_set = set()
    invalid_doc = []
    for i in range(len(df)):
        shingle = find_shingles(df.loc[i]['sequence'], k)
        if len(shingle) == 0:
            invalid_doc.append(i)
        for i in shingle:
            shingle_set.add(i)
        shingle_arr.append(shingle)
    return shingle_arr, shingle_set

shingle_arr,shingle_set = doc_shingles(df, k)     
#len(df), len(shingle_arr), len(shingle_set), invalid_doc

In [ ]:
import numpy as np
def gen_matrix(df,shingle_set,shingle_arr):
    """
    <h5>Once we have shingled our documents, we proceed ahead to make our matrix. This function takes in the shingled set and array and generates matrix from them.
    <br><br>
    It finally will return the generated matrix.</h5>
    """
    d = dict()
    for shingle in shingle_set:
        d[shingle]=list()
    for i in range(len(df)):
        doc_shingle = shingle_arr[i]
        for global_shingle in shingle_set:
            if global_shingle in doc_shingle:
                d[global_shingle].append(1)
                #print(global_shingle, d[global_shingle])
            else:
                d[global_shingle].append(0)
    mat = []
    for i in d.keys():
        mat.append(d[i])
    mat = np.array(mat)
    return mat, doc_shingle

matrix, doc_shingle= gen_matrix(df,shingle_set,shingle_arr)
"""The variable that stores our matrix after the shingling is complete. 
We later on use this to generate our signature matrix."""
matrix.shape

(1247, 4380)

In [ ]:
n_perm = 50
"""Our global variable to store the number of permutations of the matrix that we will use to convert the matrix M into signature matrix.
This also defines the size of our signature matrix."""    
def gen_permute(n, matrix):
    """
    <h5>
    <ul>
    <li>We generate the permutation matrix for minhashing using the number of permutations given to us.<br></li>
    <li>By default this value is 50, and we can change this asd required.<br></li>
    <li>It will return us the permutations array consisting of the various permutations that we will work with upon the matrix M during minHashing.</li></ul>
    </h5>
    """
    perm_arr = []
    for i in range(n_perm):
        a = np.arange(len(matrix))
        random.shuffle(a)
        perm_arr.append(a)
    return perm_arr

permute_array = gen_permute(n_perm, matrix)
"""Our global variable to store the permutations that we will work upon to convert the matrix M into signature matrix"""
len(permute_array), len(permute_array[0])

(50, 1247)

In [ ]:
def find(indx):
    """
    <h5>
    A function to locate an index within our signature matrix . 
    </h5>
    """
    indx = np.array(indx).argsort()
    sig = np.array([float('inf') for _ in range(len(mat[0]))])
    for i in range(len(indx)):
        matches = [ind for ind in range(len(mat[indx[i]])) if mat[indx[i]][ind] == 1]
        for index in matches:
            sig[index] = min(i, sig[index])
    return sig

In [ ]:
def gen_sig_matrix(perm_arr,mat):
    """
    <h5> <ul>A function to generate the signature matrix
    <li>For each row in permutation array, we take in the given set of rows from matrix M and generate an array.</li>
    <li>We proceed to hash it and keep the minimum of the hash values for each row.</li>
    <li>Finally , we append all these rows to signature array and return it as the completed signature matrix.</li>
    </ul>
    </h5>
    """
    sig_mat = list()
    l = 0
    for indx in perm_arr:
        indx = np.array(indx).argsort()
        sig = np.array([float('inf') for _ in range(len(mat[0]))])
        for i in range(len(indx)):
            matches = [ind for ind in range(len(mat[indx[i]])) if mat[indx[i]][ind] == 1]
            for index in matches:
                sig[index] = min(i, sig[index])
        if l%10 == 0:
            print(l)
        l+=1
        sig_mat.append(sig)
    return sig_mat, perm_arr
    
signature_mat, perm_arr = gen_sig_matrix(permute_array, matrix)
"""Our variable to store the signature matrix. We get this by using the permutation array on our matrix M"""

0
10
20
30
40


'Our variable to store the signature matrix. We get this by using the permutation array on our matrix M'

In [ ]:
rows = 3    #num of rows per band
"""Our global variable to store the number of rows for each band"""
n_buckets = 15  #number of buckets
"""The global variable to store the number of maximum buckets we will sort the strings into"""

def lsh(sig_mat,r,n):
    """
    <h5>
    The function for implement LSH ( Locality Sensitive Hashing ) :
    <ul>
    <li>We use the signature matrix as our input here.</li>
    <li> We run a hash function , which is the sum of the numbers to hash our values into multiple buckets </li>
    <li> Finally , we return the buckets to our user , which contain the list of possible candidate pairs which can be similar</li>
    </ul>
    </h5>
    """
    sig_mat = np.array(sig_mat)
    buckets = [[] for _ in range(n)]
    summ = []
    for i in range(0, len(sig_mat)-r+1, r):
        band = sig_mat[i:i+r]
        for doci in range(len(band[0])):
            try:
                buckets[int(np.sum(band[:, doci])%len(buckets))].append(doci)
            except:
                print("invalid doc:", doci)
    return buckets
buckets = lsh(signature_mat,rows,n_buckets)


In [ ]:
#ATGCCCCAACTAAATACTACCGTATGGCCCACCATAATTACCCCCA
query = df.iloc[0]["sequence"]
"""Our query , for which we search in the dataset """
q_class = df.iloc[0]["class"]
"""Our query class, to which our given query belongs to """
shingles = find_shingles(query, k)
query, q_class

('ATGCCCCAACTAAATACTACCGTATGGCCCACCATAATTACCCCCATACTCCTTACACTATTCCTCATCACCCAACTAAAAATATTAAACACAAACTACCACCTACCTCCCTCACCAAAGCCCATAAAAATAAAAAATTATAACAAACCCTGAGAACCAAAATGAACGAAAATCTGTTCGCTTCATTCATTGCCCCCACAATCCTAG',
 4)

In [ ]:
def similarity(s1, s2, k):
    """
    <h5> A function to compare 2 given k shingles. We get the jaccard similarity score of the two here </h5>
    """
    s1 = find_shingles(s1, k)
    s2 = find_shingles(s2, k)
    return len(s1&s2)/len(s1|s2)

In [ ]:
top_n = 5
"""We define this variable to store the number of top results that we would give as output."""
def search(query, q_class, doc_shingle, perm_arr, r, top_n):
    """
    <h5>
    A function that will take in the <ul>
    <li>The search query</li><li>The search query class</li>
    <li>Number of maximum relevant results to be shown</li></ul>
     and then print the output results , along with their similarity and the string at which they were found.</h5>
    """
    query_d = dict()
    """Generate the shingles of our query string"""
    for shingle in shingle_set:
        query_d[shingle]=list()
    #Generate the shingles of our query string  
    for global_shingle in shingle_set:
        if global_shingle in doc_shingle:
            query_d[global_shingle].append(1)
        else:
            query_d[global_shingle].append(0)
    query_mat = []
    """Generate the matrix for our given query string"""
    for i in query_d.keys():
        query_mat.append(query_d[i])
    query_mat = np.array(query_mat)
    #Generate the matrix M for our given query
    query_sig_mat = list()
    """Signature matrix of our query"""
    for indx in perm_arr:
        indx = np.array(indx).argsort()
        sig = np.array([float('inf') for _ in range(len(query_mat[0]))])
        for i in range(len(indx)):
            matches = [ind for ind in range(len(query_mat[indx[i]])) if query_mat[indx[i]][ind] == 1]
            for index in matches:
                sig[index] = min(i, sig[index])
        query_sig_mat.append(sig)    
    #Successfully generated signature matrix 
    result_bucket = []
    """The buckets for our result"""
    for i in range(0, len(query_sig_mat)-r+1, r):
        band = query_sig_mat[i:i+r]
        result_bucket.append(sum(sum(band))%len(buckets))
    result_doc = []
    for i in result_bucket:
        for j in buckets[int(i)]:
            result_doc.append([similarity(df.iloc[j]["sequence"], query, k), j])
    result = np.argsort([i[0] for i in result_doc])[::-1][:top_n]
    #Generated the result. Now we move to printing them.
    print("Query:   ", query[:80], "\tClass:", q_class)
    print()
    for i in result:
        print("Similarity:", result_doc[i][0], "\tClass:", df.iloc[result_doc[i][1]]['class'], "Doc ID: ", result_doc[i][1])
        print("Document:", df.iloc[result_doc[i][1]]['sequence'][:100])
        print()

search(query, q_class, doc_shingle, perm_arr, rows, top_n)

Query:    ATGCCCCAACTAAATACTACCGTATGGCCCACCATAATTACCCCCATACTCCTTACACTATTCCTCATCACCCAACTAAA 	Class: 4

Similarity: 1.0 	Class: 4 Doc ID:  0
Document: ATGCCCCAACTAAATACTACCGTATGGCCCACCATAATTACCCCCATACTCCTTACACTATTCCTCATCACCCAACTAAAAATATTAAACACAAACTACC

Similarity: 1.0 	Class: 4 Doc ID:  0
Document: ATGCCCCAACTAAATACTACCGTATGGCCCACCATAATTACCCCCATACTCCTTACACTATTCCTCATCACCCAACTAAAAATATTAAACACAAACTACC

Similarity: 1.0 	Class: 4 Doc ID:  0
Document: ATGCCCCAACTAAATACTACCGTATGGCCCACCATAATTACCCCCATACTCCTTACACTATTCCTCATCACCCAACTAAAAATATTAAACACAAACTACC

Similarity: 1.0 	Class: 4 Doc ID:  0
Document: ATGCCCCAACTAAATACTACCGTATGGCCCACCATAATTACCCCCATACTCCTTACACTATTCCTCATCACCCAACTAAAAATATTAAACACAAACTACC

Similarity: 1.0 	Class: 4 Doc ID:  0
Document: ATGCCCCAACTAAATACTACCGTATGGCCCACCATAATTACCCCCATACTCCTTACACTATTCCTCATCACCCAACTAAAAATATTAAACACAAACTACC

